In [95]:
import sys
sys.path.append('/home/koreyr/scripts/')

In [96]:
import numpy as np
import matplotlib.pyplot as plt
import MDAnalysis as mda 
import mdtraj as md
import pandas as pd

In [97]:
from traj_calc import *

In [86]:
def rsa(trj):
    vol_ = [121.0,265.0,187.0,187.0,148.0,214.0,214.0,97.0,216.0,
                    195.0,191.0,230.0,203.0,228.0,154.0,143.0,163.0,264.0,255.0]
    res_ =  ['A','R','N','D','C','E','Q','G','H','I','L','K','M','F','P','S','T','W','Y','V']
    rsa_dic_ = dict(zip(res_,vol_))
    CAindex = trj.topology.select('name CA')
    return np.array([rsa_dic_[trj.topology.atom(i).residue.code] for i in CAindex])

In [9]:
u = mda.Universe('../../aFUS/prod.all.gro','../../aFUS/prod.trr')

In [10]:
u

<Universe with 34403 atoms>

In [24]:
s_wat = u.select_atoms('name OW and resname SOL')

In [26]:
rdf = mda.analysis.rdf.InterRDF(s_wat)

AttributeError: module 'MDAnalysis' has no attribute 'analysis'

In [80]:
edits,bonds = md.load('../../aFUS/protein.ff.gro').topology.to_dataframe()
for i,j in zip(["NGLY", "CGLN"],["GLY","GLN"]):
    edits.loc[edits["resName"] == i, "resName"] = j
top = md.Topology.from_dataframe(edits, bonds)
traj = md.load_xtc('../../aFUS/prod.xtc',top)

In [81]:
traj

<mdtraj.Trajectory with 28407 frames, 624 atoms, 44 residues, and unitcells at 0x7f215ff72380>

In [91]:
sasa_a = md.shrake_rupley(traj,mode='residue')

In [83]:
sasa_total = sasa_a.sum(axis=1)

In [84]:
traj.topology.to_fasta()

['GMASNDYTQQATQSYGAYPTQPGQGYSQQSSQPYGQQSYSGYSQ']

In [87]:
rsa(traj)

array([ 97., 203., 121., 143., 187., 187., 255., 163., 214., 214., 121.,
       163., 214., 143., 255.,  97., 121., 255., 154., 163., 214., 154.,
        97., 214.,  97., 255., 143., 214., 214., 143., 143., 214., 154.,
       255.,  97., 214., 214., 143., 255., 143.,  97., 255., 143., 214.])

In [51]:
count = 0
sasa_res = []
for i in [len(list(traj.topology.residue(i).atoms)) for i in range(0,traj.n_residues)]:
    sasa_res.append(sasa_a[:,count:count+i].sum(1))
    count+=i
sasa_res = np.stack(sasa_res,axis=1)


In [93]:
sasa_res

array([[0.        , 1.3029892 , 0.5567284 , ..., 1.2267051 , 0.01897375,
        0.33144885],
       [0.32824498, 1.9347925 , 0.9906419 , ..., 0.8604887 , 0.79199976,
        1.6150889 ],
       [0.17476895, 1.9053276 , 0.5798737 , ..., 1.8185761 , 0.57112515,
        0.10929611],
       ...,
       [1.2489336 , 1.2366097 , 0.38747823, ..., 1.6780525 , 0.720007  ,
        2.6606364 ],
       [1.4323264 , 1.9690346 , 0.7023541 , ..., 0.72832227, 0.37746626,
        1.6884302 ],
       [1.0673987 , 1.4967836 , 0.9845367 , ..., 1.1160011 , 1.2797939 ,
        1.8354604 ]], dtype=float32)

In [94]:
sasa_a

array([[0.        , 1.3029892 , 0.55672836, ..., 1.226705  , 0.01897375,
        0.33144885],
       [0.328245  , 1.9347925 , 0.9906419 , ..., 0.8604887 , 0.79199976,
        1.6150889 ],
       [0.17476895, 1.9053276 , 0.5798736 , ..., 1.8185761 , 0.5711251 ,
        0.10929611],
       ...,
       [1.2489338 , 1.2366097 , 0.38747823, ..., 1.6780525 , 0.720007  ,
        2.6606364 ],
       [1.4323264 , 1.9690346 , 0.7023541 , ..., 0.7283224 , 0.37746626,
        1.6884301 ],
       [1.0673987 , 1.4967836 , 0.98453677, ..., 1.1160011 , 1.2797939 ,
        1.8354603 ]], dtype=float32)